# OPENCV - ResNet50

In [1]:

import cv2
import numpy as np

path_image = "/Users/lua/wild_IA/projet_fil_rouge/food-101/data/food-101/images/churros/45579.jpg"
# Charger une image
image = cv2.imread(path_image)
image = cv2.resize(image, (224, 224))  # Redimensionner pour le modèle
image = image / 255.0  # Normalisation
image = np.expand_dims(image, axis=0)  # Ajouter une dimension pour le batch


[ WARN:0@0.096] global loadsave.cpp:268 findDecoder imread_('/Users/lua/wild_IA/projet_fil_rouge/food-101/data/food-101/images/churros/45579.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.11.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [2]:
# Télécharger le fichier de classes ImageNet
imagenet_classes_url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json" # animaux (trouver celui des aliments)
#https://gist.githubusercontent.com/kklemon/3d16a10202b2784a6dfc0c7678f4f582/raw/b4278728b144a0584d4930205a5a9faad06235bc/imagenet_class_labels.txt
import requests

response = requests.get(imagenet_classes_url)
imagenet_classes = response.json()




In [3]:
print(imagenet_classes[643])

mask


In [4]:
from tensorflow.keras.applications import ResNet50

model = ResNet50(weights='imagenet')  # Chargement du modèle pré-entraîné


2025-02-14 09:23:40.720432: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
predictions = model.predict(image)
class_id = np.argmax(predictions)
print(f"Class ID prédit : {class_id}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Class ID prédit : 111


In [6]:
# Vérifier la classe correspondant à class_id = 644
predicted_label = imagenet_classes[class_id-1]
print(f"La prédiction pour class_id {class_id} est : {predicted_label}")

La prédiction pour class_id 111 est : flatworm


### Avec EfficientNetB0
(ça donne tjrs le même résultat (644 == match)) ???

In [15]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import load_model

# Charger un modèle préentraîné ou personnalisé
model = EfficientNetB0(weights='imagenet', include_top=True)

# Prédire
predictions = model.predict(image)
class_id = np.argmax(predictions)

# Vérifier la classe correspondant à class_id = 644
predicted_label = imagenet_classes[class_id]
print(f"La prédiction pour class_id {class_id} est : {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
La prédiction pour class_id 644 est : match


In [6]:
print(class_id)

644


In [4]:
# Exemple d'association avec une base de données nutritionnelle
food_database = {
    "pomme": {"calories": 52, "protéines": 0.3, "glucides": 14, "lipides": 0.2},
    "banane": {"calories": 89, "protéines": 1.1, "glucides": 23, "lipides": 0.3}
}

detected_food = "pomme"  # Nom détecté
nutrition = food_database.get(detected_food, {})
print(f"Valeurs nutritionnelles pour {detected_food}: {nutrition}")


Valeurs nutritionnelles pour pomme: {'calories': 52, 'protéines': 0.3, 'glucides': 14, 'lipides': 0.2}


# MODEL YOLOv8

In [6]:
from ultralytics import YOLO

# Charger un modèle pré-entraîné (par exemple YOLOv8n pour la version nano)
model = YOLO('yolov8s.pt')


In [7]:
# Charger une image locale
results = model('/Users/lua/wild_IA/projet_fil_rouge/food-101/data/food-101/images/waffles/3138.jpg')

# Parcourir les résultats et afficher chaque image avec ses annotations
for result in results:
    result.plot()  # Méthode correcte pour afficher l'image avec les détections



image 1/1 /Users/lua/wild_IA/projet_fil_rouge/food-101/data/food-101/images/waffles/3138.jpg: 448x640 1 pizza, 1 dining table, 269.4ms
Speed: 9.3ms preprocess, 269.4ms inference, 12.5ms postprocess per image at shape (1, 3, 448, 640)


In [32]:
# Parcourir les détections
for result in results:
    for box in result.boxes:
        print(f"Classe : {box.cls}, Score : {box.conf}, Coordonnées : {box.xyxy}")


Classe : tensor([54.]), Score : tensor([0.9324]), Coordonnées : tensor([[ 15.8740,  73.5371, 237.7036, 325.0114]])
Classe : tensor([54.]), Score : tensor([0.8543]), Coordonnées : tensor([[186.5239,  63.9664, 495.5538, 383.8908]])
